In [6]:
import pandas as pd


df = pd.read_csv(r'C:\Users\ISS-User1\Documents\Eugene\Glowing-Torch\datasets\custom_dataset.csv', encoding='latin1')
df.head()

,name,category
0,METAL GEAR Archive UT (Short Sleeve Graphic T-...,Other T-Shirts
1,Wireless Bra (3D Hold),None of the given categories
2,Wireless Bra (Ultra Relax),None of the given categories
3,Wireless Bra (Ultra Relax) (With AIRism),None of the given categories
4,Wireless Bra (Stretch Lace),None of the given categories


In [4]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')
stop = stopwords.words('english')
print(stop[::10])

porter = PorterStemmer()

[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     An existing connection was forcibly closed by the
[nltk_data]     remote host>


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\ISS-User1/nltk_data'
    - 'c:\\Users\\ISS-User1\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\ISS-User1\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\ISS-User1\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\ISS-User1\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
import re

def preprocess_data(text):
    ''' The function to remove punctuation,
    stopwords and apply stemming'''
    
    words = re.sub("[^a-zA-Z]", " ", text)
    words = [word.lower() for word in text.split() if word.lower() not in stop]
    words = [porter.stem(word) for word in words]
    return " ".join(words)

df['name'] = df['name'].apply(preprocess_data)


In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

tokens = df['name'][0]

nltk.pos_tag(word_tokenize(tokens))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline


vectorizer = CountVectorizer()
vectorizer.fit(df['description'])
vector = vectorizer.transform(df['description'])

In [ ]:
tfidf_converter = TfidfTransformer()
X_tfidf = tfidf_converter.fit_transform(vector).toarray()
X_tfidf

In [ ]:
X = df['name']
y = df['category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
import numpy as np
from sklearn.svm import LinearSVC

svc = Pipeline([('vect', CountVectorizer(min_df=5, ngram_range=(1,2))),
               ('tfidf', TfidfTransformer()),
               ('model',LinearSVC()),
               ])

svc.fit(X_train, y_train)

ytest = np.array(y_test)
y_pred = svc.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(ytest, y_pred))